In [10]:
import ffn
%matplotlib inline
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import pickle
import talib
from talib import MA_Type
import requests
import pickle

In [11]:
# 計算 MaxDD
def DrawDownAnalysis(cumRet):
    dd_series = ffn.core.to_drawdown_series(cumRet)
    dd_details = ffn.core.drawdown_details(dd_series)
    return dd_details['drawdown'].min(), dd_details['days'].max()

In [12]:
# 利用策略產生的持有部位資訊，計算底下四個指標來判斷投資績效
# sharpe ratio: 判斷報酬的好壞跟穩定度，數值越大越好
# maxdd: maximum drawdown, 最糟糕的狀況會賠幾 %
# maxddd: maximum drawdown duration, 低於上一次最高報酬的天數
# cumRet[-1]: 最後賺的 % 數

def indicators(df):
    dailyRet = df['Close'].pct_change()
    excessRet = (dailyRet - 0.04/252)[df['positions'] == 1]
    SharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)
    
    cumRet = np.cumprod(1+excessRet)
    
    maxdd, maxddd = DrawDownAnalysis(cumRet)
    
    return SharpeRatio, maxdd, maxddd, cumRet[-1]

In [13]:
def 老師的RSI(df):
    df['RSI'] = talib.RSI(df['Close'].values)
    
    has_position = False
    df['signals'] = 0
    for t in range(2, df['signals'].size):
        if df['RSI'][t-1] < 30:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['RSI'][t-1] > 70:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()
    return df

# 多頭排列策略
- 概念：依照多頭排列代表多頭市場，依照道氏理論，趨勢會繼續持續直到反轉
- 進場策略：MA5 > MA10 > MA20
    
- 出場策略：收盤價小於MA5且MA5小於MA20
    


In [14]:
# 定義你的策略

def 多頭排列策略(df):
    df['5MA']=np.round(pd.Series.rolling(df['Close'], window=5).mean(), 2)
    df['10MA']=np.round(pd.Series.rolling(df['Close'], window=10).mean(), 2)
    df['20MA']=np.round(pd.Series.rolling(df['Close'], window=20).mean(), 2)


    df['UBB'], df['MBB'], df['LBB'] = talib.BBANDS(df['Close'].values, matype=MA_Type.T3)

    has_position = False
    df['signals'] = 0

    for t in range(2, df['signals'].size):
        if df['5MA'][t] > df['10MA'][t] and df['10MA'][t] > df['20MA'][t]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['Close'][t] < df['5MA'][t] and df['5MA'][t] <df['10MA'][t]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False
    df['positions'] = df['signals'].cumsum().shift()

    return df

# 布林通道突破策略
- 進場策略：當價格大於布林通道上緣 且 5MA 大於 10MA 
- 出場策略：當價格小於布林通道中線

In [23]:
def 不靈通道有沒有靈(df):
    df['UBB'], df['MBB'], df['LBB'] = talib.BBANDS(df['Close'].values, matype=MA_Type.T3)
    df['60MA']=np.round(pd.Series.rolling(df['Close'], window=60).mean(), 2)
    df['120MA']=np.round(pd.Series.rolling(df['Close'], window=120).mean(), 2)
    df['Vol10MA']=np.round(pd.Series.rolling(df['Volume'], window=10).mean(), 2)
    df['RSI'] = talib.RSI(df['Close'].values)
    df['SAR']=talib.SAR(df['High'].values,df['Low'].values, acceleration=0.02, maximum=0.2)
    
    has_position = False
    df['signals'] = 0
    
    for t in range(2, df['signals'].size):
        if df['Close'][t] > df['UBB'][t] and df['5MA'][t] > df['10MA'][t]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        elif df['Close'][t] < df['MBB'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()
    return df

In [24]:
import os
os.chdir('/Users/alex/Documents/GitHub/M064810011_Quant/金融投資Midterm')
with open('stockdata','rb') as f:
    data=pickle.load(file=f)

In [25]:
def apply_strategy(strategy, df):
    return strategy(df)

In [26]:
# 計算各支股票的回測結果
results = []

#strategies = [Breakout_strategy, RSI_7030_strategy, BBands_strategy, 學號_strategy]
strategies = [多頭排列策略,老師的RSI,不靈通道有沒有靈]

for symbol in data:
    for strategy in strategies:
        try:
            apply_strategy(strategy, data[symbol])
            if np.all(data[symbol]['signals']==0):
                print("Symbol:", symbol, "使用", strategy.__name__, "策略沒有出現買賣訊號。")
                continue
            SharpeRatio, maxdd, maxddd, finalRet = indicators(data[symbol])
            days = (data[symbol].index[-1] - data[symbol].index[0]).days
            results.append((SharpeRatio, maxdd, maxddd, finalRet, days,
                            data[symbol][data[symbol]['signals'] > 0]['signals'].sum(), symbol, strategy.__name__))
        except Exception as e:
            print("Error occurs at symbol:", symbol, "Strategy:", strategy.__name__, "==>", e.args)

In [27]:
results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns', 
                                            'days', 'entries','symbol','strategy'])

In [28]:
results_df.head()

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
0,-0.280316,-0.196293,281,0.918100,705,17,TICC,多頭排列策略
1,0.395515,-0.244817,482,1.077756,705,2,TICC,老師的RSI
2,0.328546,-0.149465,344,1.020922,705,14,TICC,不靈通道有沒有靈
3,0.473014,-0.353410,313,1.107711,705,20,TWIN,多頭排列策略
4,2.424484,-0.169115,115,2.151881,705,3,TWIN,老師的RSI


In [29]:
results_df.sort_values('MaxDrawDown',ascending=False).head()

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
55,6.306176,-0.003654,122,1.028222,473,3,LCA,老師的RSI
13,10.243474,-0.006251,3,1.132402,705,1,GOOG,老師的RSI
29,4.049327,-0.020559,57,1.052255,203,4,CSML,不靈通道有沒有靈
27,3.439969,-0.020969,88,1.103369,203,4,CSML,多頭排列策略
49,-1.279635,-0.029044,62,0.982055,384,1,SCAC,老師的RSI


## 差點就贏老師了，但是我的策略進場次數比較高

In [32]:
results_df.sort_values('returns',ascending=False).head(10)

,sharpe,MaxDrawDown,MaxDrawDownDuration,returns,days,entries,symbol,strategy
7,2.013777,-0.219296,167,2.477803,705,4,AMOT,老師的RSI
4,2.424484,-0.169115,115,2.151881,705,3,TWIN,老師的RSI
32,1.727600,-0.299653,191,1.897584,705,13,NLNK,不靈通道有沒有靈
18,1.977698,-0.161377,154,1.767895,705,16,RICK,多頭排列策略
5,2.751975,-0.144159,241,1.744890,705,18,TWIN,不靈通道有沒有靈
79,1.060919,-0.390808,329,1.462185,705,5,VRA,老師的RSI
76,2.848694,-0.080343,320,1.429350,705,3,NKSH,老師的RSI
58,6.525037,-0.049054,66,1.417397,705,2,BBGI,老師的RSI
80,3.025028,-0.111450,249,1.413310,705,10,VRA,不靈通道有沒有靈
30,1.058476,-0.413177,173,1.412551,705,11,NLNK,多頭排列策略
